In [3]:
# Data and NBA API
import pandas as pd
import numpy as np

from scipy.stats import norm, gaussian_kde, percentileofscore

pd.options.display.max_columns = None
from nba_api.stats.static import players
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.endpoints import playercareerstats

# For Shot Chart
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import cm
from matplotlib.patches import Circle, Rectangle, Arc, ConnectionPatch
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import LinearSegmentedColormap, ListedColormap, BoundaryNorm
from matplotlib.path import Path
from matplotlib.patches import PathPatch

sns.set_style('white')
sns.set_color_codes()

In [4]:
def get_player_shotchartdetail(player_name, season_id):
    """
    Parameters
    ----------
    player_name: name of the player with Capital
    season_id: ex. 2012-13
    """
    
    # player dictionary
    nba_players = players.get_players()
    player_dict = [player for player in nba_players if player['full_name'] == player_name][0]
    
    # career df
    career = playercareerstats.PlayerCareerStats(player_id=player_dict['id'])
    career_df = career.get_data_frames()[0]
    
    # team id during the season
    team_id = career_df[career_df['SEASON_ID'] == season_id]['TEAM_ID']
    
    # shotchardtdetail endpoint
    shotchartlist = shotchartdetail.ShotChartDetail(team_id=int(team_id), 
                                                   player_id=int(player_dict['id']), 
                                                   season_type_all_star='Regular Season', 
                                                   season_nullable=season_id,
                                                   context_measure_simple="FGA").get_data_frames()
    
    return shotchartlist[0], shotchartlist[1]

In [19]:
player_dict = {'Damian Lillard': '2019-20', 'Joel Embiid': '2019-20', 'Ben Simmons': '2019-20'}



def extract_players_API(dict_of_player_and_season):
    '''
    This function takes in a dictionary of players & years and outputs a dictionary of
    player name as the key & their pandas df as the value. This means that we can now
    build out the whole project directly from the API!!!
    '''
    
    dict_of_players_pd_df = {}
    
    for player in dict_of_player_and_season.keys():
        player_shot_chart_df, league_avg = get_player_shotchartdetail(player, dict_of_player_and_season[player])
        dict_of_players_pd_df[player] = player_shot_chart_df
        
    return dict_of_players_pd_df

In [16]:
dame_joel_simmons = extract_players_API(player_dict)

dame_joel_simmons['Joel Embiid']

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0021900008,11,203954,Joel Embiid,1610612755,Philadelphia 76ers,1,11,26,Made Shot,Alley Oop Layup shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,1,-17,5,1,1,20191023,PHI,BOS
1,Shot Chart Detail,0021900008,16,203954,Joel Embiid,1610612755,Philadelphia 76ers,1,10,55,Missed Shot,Fadeaway Jump Shot,2PT Field Goal,Mid-Range,Center(C),8-16 ft.,15,14,157,1,0,20191023,PHI,BOS
2,Shot Chart Detail,0021900008,19,203954,Joel Embiid,1610612755,Philadelphia 76ers,1,10,34,Missed Shot,Turnaround Fadeaway shot,2PT Field Goal,Mid-Range,Left Side(L),8-16 ft.,10,-106,1,1,0,20191023,PHI,BOS
3,Shot Chart Detail,0021900008,150,203954,Joel Embiid,1610612755,Philadelphia 76ers,1,1,19,Missed Shot,Jump Shot,3PT Field Goal,Above the Break 3,Center(C),24+ ft.,25,-25,258,1,0,20191023,PHI,BOS
4,Shot Chart Detail,0021900008,159,203954,Joel Embiid,1610612755,Philadelphia 76ers,1,0,14,Missed Shot,Driving Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,3,25,23,1,0,20191023,PHI,BOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,Shot Chart Detail,0021901315,164,203954,Joel Embiid,1610612755,Philadelphia 76ers,1,0,7,Missed Shot,Turnaround Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,13,-54,124,1,0,20200814,HOU,PHI
797,Shot Chart Detail,0021901315,242,203954,Joel Embiid,1610612755,Philadelphia 76ers,2,7,0,Made Shot,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),8-16 ft.,14,-146,11,1,1,20200814,HOU,PHI
798,Shot Chart Detail,0021901315,305,203954,Joel Embiid,1610612755,Philadelphia 76ers,2,0,43,Made Shot,Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,12,19,1,1,20200814,HOU,PHI
799,Shot Chart Detail,0021901315,336,203954,Joel Embiid,1610612755,Philadelphia 76ers,3,9,35,Missed Shot,Layup Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,6,20,59,1,0,20200814,HOU,PHI


In [12]:
dame_joel_simmons['Damian Lillard']

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0021900013,22,203081,Damian Lillard,1610612757,Portland Trail Blazers,1,10,25,Missed Shot,Driving Floating Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,5,58,-5,1,0,20191023,POR,DEN
1,Shot Chart Detail,0021900013,46,203081,Damian Lillard,1610612757,Portland Trail Blazers,1,8,42,Missed Shot,Jump Shot,3PT Field Goal,Left Corner 3,Left Side(L),24+ ft.,23,-232,-9,1,0,20191023,POR,DEN
2,Shot Chart Detail,0021900013,145,203081,Damian Lillard,1610612757,Portland Trail Blazers,1,2,44,Made Shot,Driving Dunk Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,-3,8,1,1,20191023,POR,DEN
3,Shot Chart Detail,0021900013,295,203081,Damian Lillard,1610612757,Portland Trail Blazers,2,4,22,Missed Shot,Pullup Jump shot,3PT Field Goal,Above the Break 3,Center(C),24+ ft.,24,-52,244,1,0,20191023,POR,DEN
4,Shot Chart Detail,0021900013,305,203081,Damian Lillard,1610612757,Portland Trail Blazers,2,3,8,Made Shot,Pullup Jump shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,19,-105,159,1,1,20191023,POR,DEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1344,Shot Chart Detail,0021901309,554,203081,Damian Lillard,1610612757,Portland Trail Blazers,4,9,32,Made Shot,Step Back Jump shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,26,-119,236,1,1,20200813,BKN,POR
1345,Shot Chart Detail,0021901309,560,203081,Damian Lillard,1610612757,Portland Trail Blazers,4,8,31,Missed Shot,Driving Floating Bank Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,5,-46,38,1,0,20200813,BKN,POR
1346,Shot Chart Detail,0021901309,581,203081,Damian Lillard,1610612757,Portland Trail Blazers,4,7,12,Made Shot,Jump Shot,3PT Field Goal,Above the Break 3,Center(C),24+ ft.,27,65,268,1,1,20200813,BKN,POR
1347,Shot Chart Detail,0021901309,612,203081,Damian Lillard,1610612757,Portland Trail Blazers,4,4,34,Missed Shot,Pullup Jump shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,29,-149,249,1,0,20200813,BKN,POR


In [18]:
dame_joel_simmons['Ben Simmons']

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0021900008,34,1627732,Ben Simmons,1610612755,Philadelphia 76ers,1,9,20,Made Shot,Alley Oop Layup shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,-24,5,1,1,20191023,PHI,BOS
1,Shot Chart Detail,0021900008,54,1627732,Ben Simmons,1610612755,Philadelphia 76ers,1,7,38,Made Shot,Driving Finger Roll Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,1,-16,8,1,1,20191023,PHI,BOS
2,Shot Chart Detail,0021900008,284,1627732,Ben Simmons,1610612755,Philadelphia 76ers,2,4,59,Made Shot,Hook Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,6,-41,50,1,1,20191023,PHI,BOS
3,Shot Chart Detail,0021900008,294,1627732,Ben Simmons,1610612755,Philadelphia 76ers,2,4,32,Missed Shot,Turnaround Hook Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,4,42,22,1,0,20191023,PHI,BOS
4,Shot Chart Detail,0021900008,352,1627732,Ben Simmons,1610612755,Philadelphia 76ers,2,1,21,Made Shot,Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,1,-17,-3,1,1,20191023,PHI,BOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,Shot Chart Detail,0021901263,93,1627732,Ben Simmons,1610612755,Philadelphia 76ers,1,4,28,Missed Shot,Turnaround Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,7,-23,74,1,0,20200805,WAS,PHI
643,Shot Chart Detail,0021901263,202,1627732,Ben Simmons,1610612755,Philadelphia 76ers,2,8,55,Missed Shot,Floating Jump shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,13,21,130,1,0,20200805,WAS,PHI
644,Shot Chart Detail,0021901263,339,1627732,Ben Simmons,1610612755,Philadelphia 76ers,3,11,45,Missed Shot,Driving Floating Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,5,3,59,1,0,20200805,WAS,PHI
645,Shot Chart Detail,0021901263,362,1627732,Ben Simmons,1610612755,Philadelphia 76ers,3,9,56,Missed Shot,Driving Finger Roll Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,3,26,1,0,20200805,WAS,PHI
